In [1]:
import pandas as pd
import os
import glob
import numpy as np

In [2]:
code_dir = os.getcwd()
base_dir = code_dir.replace("/Z_codes_local", "")

In [13]:
def mouse_to_human(inList, reverse=False):
    inList = [str(x).upper() for x in inList]
    
    mmhs_ref = base_dir + "/Human_to_Mouse_geneNames.csv"
    mmhs_df = pd.read_csv(mmhs_ref)
    
    
    if not reverse:
        old_names = mmhs_df['mouse_geneName'].values.tolist()
        new_names = mmhs_df['human_geneName'].values.tolist()
    else:
        old_names = mmhs_df['human_geneName'].values.tolist()
        new_names = mmhs_df['mouse_geneName'].values.tolist()
    old_names = [ str(x).upper() for x in old_names]
    new_names = [ str(x).upper() for x in new_names]
    
    outList = []
    for i in inList:
        if i in old_names:
            idx = old_names.index(i)
            outList.append(new_names[idx])
        else:
            outList.append("")
    if reverse:
        outList = [x.title() for x in outList]
        return(outList)
    else:
        return(outList)

## 0. Convert mouse references to human

*Human to mouse*

In [14]:
input_dir = base_dir + '/X_Formatted_csv/*'
output_dir = base_dir + '/X_GeneSignatures_mm'

sig_files = glob.glob("%s/*signatures.csv"%input_dir)
for i in sig_files:
    if not os.path.exists(i.replace(".csv", "_mouse.csv")):
        print(i)
        i_mouse = i.replace(".csv", "_mouse.csv")
        i_df = pd.read_csv(i)
        i_df['gene_symbol'] = mouse_to_human(i_df['gene_symbol'].values.tolist(), reverse=True)
        i_df = i_df[i_df['gene_symbol'] != '']
        i_df.to_csv(i_mouse, index=False)        

/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_Formatted_csv/CD4/2013_IMMUNITY_Locci_signatures.csv
/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_Formatted_csv/CD4/2015_NAT_IMMUNOL_Choi_signatures.csv
/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_Formatted_csv/CD4/2018_CELL_Schmiedel_CD4_signatures.csv
/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_Formatted_csv/CD4/2018_CELL_Schmiedel_allCellTypes_signatures.csv
/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_Formatted_csv/CD4/2018_SCI_IMMUNOL_Patil_signatures.csv
/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_Formatted_csv/CD4/2019_NAT_COMMU_Szabo_signatures.csv
/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_Formatted_csv/CD4/2020_IMMUNITY_Cillo_signatures.csv
/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_Formatted_csv/CD4/2019_NAT_COMMU_BIOL_Schoettler_signatures.csv
/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_Formatted_csv/CD8/2014_IMMUNITY_Gaiha_signatures.csv
/me

## 1. Summarize all references 

In [17]:
wk_dir = base_dir + "/X_GeneSignatures_mm"
os.chdir(wk_dir)

input_dir = base_dir + '/X_Formatted_csv'

all_sig_files = glob.glob("%s/*/*signatures_mouse.csv"%input_dir)
all_sig_df = pd.DataFrame()

for i in all_sig_files:
    i_basename = i.split("/")[-1].replace("_signatures_mouse.csv", "")
    i_df = pd.read_csv(i)[['gs_name', 'gene_symbol']]
    i_df['gs_name'] = [i_basename + "---" + x for x in i_df['gs_name'].values]
    all_sig_df = all_sig_df.append(i_df)

all_sig_df['gs_name'] = [x.replace(" ", "_").replace("/","-").replace("ï", "i") for x in all_sig_df['gs_name'].values]
all_sig_df_sum = all_sig_df.groupby('gs_name').count()
all_sig_df_sum.columns = ['gene_number']

In [18]:
all_sig_df.to_csv("all_mouse_T_cell_signatures.csv", index=False)
all_sig_df_sum.to_csv("all_mouse_T_cell_gs_signatures_summary.csv")

## 2. Create subset references

In [19]:
wk_dir = base_dir + "/X_GeneSignatures_mm"
os.chdir(wk_dir)

all_sig_df = pd.read_csv("all_mouse_T_cell_signatures.csv")
gs_anno = pd.read_csv('all_T_cell_gs_signatures_summary_annotated.csv')

In [20]:
gs_anno_use = gs_anno[gs_anno['use'] != 'n']
all_sig_df_use = all_sig_df[[True if x in gs_anno_use['gs_name'].values else False for x in all_sig_df['gs_name'].values]]

In [21]:
gs_anno_use_types = np.unique(gs_anno_use['type'])

In [22]:
for gs_type in gs_anno_use_types:
    gs_type_names = gs_anno_use[gs_anno_use['type'] == gs_type]['gs_name'].values
    all_sig_df_gs_type = all_sig_df_use[[True if x in gs_type_names else False for x in all_sig_df_use['gs_name'].values]]
    all_sig_df_gs_type.to_csv("%s_mm_sigs.csv"%gs_type, index=False)